# **MULTICLASS HATESPEECH STRENGTHNESS**

## **INSTALL AND IMPORT LIBRARY**

In [ ]:
!pip -q install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import pandas as pd
import re
import numpy as np
import tensorflow as tf
import pandas as pd
import shutil
from google.colab import drive
from google.colab import files
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.metrics import classification_report, accuracy_score
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
# drive.mount('/content/drive')

## **LOAD AND TRANSFORM DATASET**

In [ ]:
df = load_dataset("keelezibel/hate-speech-indo")
df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/228 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/4.41M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15127 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3782 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'hs', 'abusive', 'hs_individual', 'hs_group', 'hs_religion', 'hs_race', 'hs_physical', 'hs_gender', 'hs_other', 'hs_weak', 'hs_moderate', 'hs_strong'],
        num_rows: 15127
    })
    test: Dataset({
        features: ['text', 'hs', 'abusive', 'hs_individual', 'hs_group', 'hs_religion', 'hs_race', 'hs_physical', 'hs_gender', 'hs_other', 'hs_weak', 'hs_moderate', 'hs_strong'],
        num_rows: 3782
    })
})

In [ ]:
df_train = df['train'].to_pandas()
df_test = df['test'].to_pandas()

In [ ]:
df = pd.concat([df_train, df_test], ignore_index=True)
df.head()

,text,hs,abusive,hs_individual,hs_group,hs_religion,hs_race,hs_physical,hs_gender,hs_other,hs_weak,hs_moderate,hs_strong
0,USER USER USER USER USER USER USER USER USER U...,0,1,0,0,0,0,0,0,0,0,0,0
1,RT USER Cebong akan terus membela junjungannya...,1,1,0,1,1,0,0,0,0,0,1,0
2,USER Terus bagaimana dgn uu MD3 bang?',0,0,0,0,0,0,0,0,0,0,0,0
3,"""\xf0\x9f\x91\xa6: """"Jangan jahatin anak orang...",1,1,1,0,0,0,1,0,0,1,0,0
4,Derita nun jauh disana akan sama dirasakan per...,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18909 entries, 0 to 18908
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           18909 non-null  object
 1   hs             18909 non-null  int64 
 2   abusive        18909 non-null  int64 
 3   hs_individual  18909 non-null  int64 
 4   hs_group       18909 non-null  int64 
 5   hs_religion    18909 non-null  int64 
 6   hs_race        18909 non-null  int64 
 7   hs_physical    18909 non-null  int64 
 8   hs_gender      18909 non-null  int64 
 9   hs_other       18909 non-null  int64 
 10  hs_weak        18909 non-null  int64 
 11  hs_moderate    18909 non-null  int64 
 12  hs_strong      18909 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 1.9+ MB


## **PREPROCESSING DATA**

In [ ]:
def remove_duplicate_user_rt(text):
    words = text.split()  # Memisahkan teks menjadi daftar kata
    unique_words = []

    for word in words:
        if word in {"USER", "RT"} and (not unique_words or unique_words[-1] != word):
            unique_words.append(word)
        elif word not in {"USER", "RT"}:
            unique_words.append(word)

    return " ".join(unique_words)  # Menggabungkan kembali menjadi teks

In [ ]:
df["text"] = df["text"].apply(remove_duplicate_user_rt)

In [ ]:
def clean_text(text):
    # Hapus "USER", "RT", "\n", "www", dan "https"
    text = re.sub(r"\b(USER|RT|www|https)\b", "", text, flags=re.IGNORECASE)

    # Hapus karakter ASCII emoticon seperti "\xf0\x9f\x91\xa6"
    text = re.sub(r"\\x[a-fA-F0-9]{2}", "", text)

    # Hapus karakter whitespace tambahan
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
df["text"] = df["text"].apply(clean_text)

In [ ]:
df.head()

,text,hs,abusive,hs_individual,hs_group,hs_religion,hs_race,hs_physical,hs_gender,hs_other,hs_weak,hs_moderate,hs_strong
0,Keracunan tiap nete',0,1,0,0,0,0,0,0,0,0,0,0
1,Cebong akan terus membela junjungannya walaupu...,1,1,0,1,1,0,0,0,0,0,1,0
2,Terus bagaimana dgn uu MD3 bang?',0,0,0,0,0,0,0,0,0,0,0,0
3,""": """"Jangan jahatin anak orang""""\n: """"Baru jug...",1,1,1,0,0,0,1,0,0,1,0,0
4,Derita nun jauh disana akan sama dirasakan per...,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
def letters_only(text):
  text = re.sub(r"[^a-zA-Z]+", " ", text).strip().lower()
  return text

In [ ]:
df["text"] = df["text"].apply(letters_only)

In [ ]:
df.head()

,text,hs,abusive,hs_individual,hs_group,hs_religion,hs_race,hs_physical,hs_gender,hs_other,hs_weak,hs_moderate,hs_strong
0,keracunan tiap nete,0,1,0,0,0,0,0,0,0,0,0,0
1,cebong akan terus membela junjungannya walaupu...,1,1,0,1,1,0,0,0,0,0,1,0
2,terus bagaimana dgn uu md bang,0,0,0,0,0,0,0,0,0,0,0,0
3,jangan jahatin anak orang n baru juga sekali u...,1,1,1,0,0,0,1,0,0,1,0,0
4,derita nun jauh disana akan sama dirasakan per...,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
!wget -O kamusalay.csv https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/new_kamusalay.csv

--2024-12-07 15:39:52--  https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/new_kamusalay.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 285941 (279K) [application/octet-stream]
Saving to: ‘kamusalay.csv’

kamusalay.csv       100%[===================>] 279.24K  --.-KB/s    in 0.04s   

2024-12-07 15:39:52 (7.36 MB/s) - ‘kamusalay.csv’ saved [285941/285941]



In [ ]:
 df_slang = pd.read_csv('kamusalay.csv',header=None, encoding="latin1")
 df_slang.head()

,0,1
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [ ]:
df_slang = df_slang.rename(columns={0: "nonformal", 1: "formal"})

In [ ]:
slang_dict = {df_slang.nonformal[i]: df_slang.formal[i] for i in range(0, len(df_slang.nonformal.values))}

In [ ]:
def replace_slang(sentence):
    tokens = sentence.split()
    for i, token in enumerate(tokens):
        if token in slang_dict.keys():
            tokens[i] = slang_dict[token]
    return ' '.join(tokens)

df['text'] = df['text'].apply(replace_slang)
df.head()

,text,hs,abusive,hs_individual,hs_group,hs_religion,hs_race,hs_physical,hs_gender,hs_other,hs_weak,hs_moderate,hs_strong
0,keracunan tiap nete,0,1,0,0,0,0,0,0,0,0,0,0
1,cebong akan terus membela junjungannya walaupu...,1,1,0,1,1,0,0,0,0,0,1,0
2,terus bagaimana dengan undang undang md bang,0,0,0,0,0,0,0,0,0,0,0,0
3,jangan menjahati anak orang dan baru juga seka...,1,1,1,0,0,0,1,0,0,1,0,0
4,derita nun jauh di sana akan sama dirasakan pe...,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_strengthness = df[['text', 'hs_weak', 'hs_moderate', 'hs_strong']]

In [ ]:
df_strengthness.replace(["", "None", "null"], np.nan, inplace=True)

<ipython-input-22-e3a3eff998fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_strengthness.replace(["", "None", "null"], np.nan, inplace=True)


In [ ]:
df_strengthness.dropna(inplace=True)
df_strengthness.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18844 entries, 0 to 18908
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         18844 non-null  object
 1   hs_weak      18844 non-null  int64 
 2   hs_moderate  18844 non-null  int64 
 3   hs_strong    18844 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 736.1+ KB


<ipython-input-23-4077ce73d4fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_strengthness.dropna(inplace=True)


In [ ]:
df_strengthness.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18844 entries, 0 to 18908
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         18844 non-null  object
 1   hs_weak      18844 non-null  int64 
 2   hs_moderate  18844 non-null  int64 
 3   hs_strong    18844 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 736.1+ KB


### **Undersampling Process**

In [ ]:
df_strong = df_strengthness[df_strengthness['hs_strong']==1]
df_weak = df_strengthness[df_strengthness['hs_weak']==1]
df_moderate = df_strengthness[df_strengthness['hs_moderate']==1]

In [ ]:
df_moderate.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2385 entries, 1 to 18907
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         2385 non-null   object
 1   hs_weak      2385 non-null   int64 
 2   hs_moderate  2385 non-null   int64 
 3   hs_strong    2385 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 93.2+ KB


In [ ]:
df_weak.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4883 entries, 3 to 18906
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         4883 non-null   object
 1   hs_weak      4883 non-null   int64 
 2   hs_moderate  4883 non-null   int64 
 3   hs_strong    4883 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 190.7+ KB


In [ ]:
df_sample_strong = df_strong.sample(n=600, random_state=42)

In [ ]:
df_sample_weak = df_weak.sample(n=600, random_state=42)

In [ ]:
df_sample_moderate = df_moderate.sample(n=600, random_state=42)

In [ ]:
df_sample_moderate.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 12673 to 851
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         600 non-null    object
 1   hs_weak      600 non-null    int64 
 2   hs_moderate  600 non-null    int64 
 3   hs_strong    600 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 23.4+ KB


In [ ]:
df_sample = pd.concat([df_sample_strong, df_sample_weak, df_sample_moderate], ignore_index=True)

In [ ]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         1800 non-null   object
 1   hs_weak      1800 non-null   int64 
 2   hs_moderate  1800 non-null   int64 
 3   hs_strong    1800 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 56.4+ KB


In [ ]:
shuffled_indices = np.random.permutation(len(df_sample))

In [ ]:
shuffled_df = df_sample.iloc[shuffled_indices]

In [ ]:
shuffled_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1800 entries, 180 to 481
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         1800 non-null   object
 1   hs_weak      1800 non-null   int64 
 2   hs_moderate  1800 non-null   int64 
 3   hs_strong    1800 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 70.3+ KB


In [ ]:
shuffled_df.to_csv('shuffled_df.csv', index=False)

In [ ]:
df_strengthness_train, df_strengthness_test = train_test_split(shuffled_df, test_size=0.2, random_state=42)

In [ ]:
df_strengthness_val, df_strengthness_test = train_test_split(df_strengthness_test, test_size=0.2, random_state=42)

In [ ]:
df_strengthness_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 288 entries, 1412 to 183
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         288 non-null    object
 1   hs_weak      288 non-null    int64 
 2   hs_moderate  288 non-null    int64 
 3   hs_strong    288 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 11.2+ KB


## **TOKENIZING**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

def tokenize_text(text, max_length=500):
    """
    Tokenisasi teks menggunakan IndoBERT tokenizer dan mengembalikan token_ids dan attention_mask.
    """
    encoded = tokenizer(text.to_list(), padding=True, truncation=True, max_length=max_length, return_tensors='tf')

    token_ids = encoded['input_ids']
    attention_mask = encoded['attention_mask']

    return token_ids, attention_mask

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
strengthtrain_token_ids, strengthtrain_attention_mask = tokenize_text(df_strengthness_train['text'])
strengthtest_token_ids, strengthtest_attention_mask = tokenize_text(df_strengthness_test['text'])
strengthval_token_ids, strengthval_attention_mask = tokenize_text(df_strengthness_val['text'])

In [ ]:
label_strengthness_train = tf.convert_to_tensor(df_strengthness_train.iloc[:,1:].values, dtype=tf.float32)
label_strengthness_test = tf.convert_to_tensor(df_strengthness_test.iloc[:,1:].values, dtype=tf.float32)
label_strengthness_val = tf.convert_to_tensor(df_strengthness_val.iloc[:,1:].values, dtype=tf.float32)

In [ ]:
formatted_strengthness_train = tf.data.Dataset.from_tensor_slices(({"input_ids":strengthtrain_token_ids, "attention_mask":strengthtrain_attention_mask}, label_strengthness_train))
formatted_strengthness_test = tf.data.Dataset.from_tensor_slices(({"input_ids":strengthtest_token_ids, "attention_mask":strengthtest_attention_mask}, label_strengthness_test))
formatted_strengthness_val = tf.data.Dataset.from_tensor_slices(({"input_ids":strengthval_token_ids, "attention_mask":strengthval_attention_mask}, label_strengthness_val))

In [ ]:
batch_size = 3
formatted_strengthness_train = formatted_strengthness_train.shuffle(len(df_strengthness_train)).batch(batch_size)
formatted_strengthness_test = formatted_strengthness_test.batch(batch_size)
formatted_strengthness_val = formatted_strengthness_val.batch(batch_size)

In [ ]:
print(label_strengthness_train.shape[1])

3


## **CONNECT TO GOOGLE DRIVE**

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/DeepLearning'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_filepath = os.path.join(checkpoint_dir, "indobert_epoch_{epoch:02d}.ckpt")

In [ ]:
def save_checkpoint(model, optimizer, epoch, wait, best_val_loss, filepath):
  model.save_weights(filepath)

  with open(filepath + '.meta', 'w') as f:
    f.write(f"{epoch}\n")
    f.write(f"{wait}\n")
    f.write(f"{best_val_loss}\n")

def load_checkpoint(model, optimizer, checkpoint_dir):
  latest_ckpt = tf.train.latest_checkpoint(checkpoint_dir)
  if latest_ckpt:
    print(f"memuat checkpoint dari {latest_ckpt}")
    model.load_weights(latest_ckpt)

    with open(latest_ckpt + '.meta', 'r') as f:
      lines = f.readlines()
      initial_epoch = int(lines[0].strip())
      wait = int(lines[1].strip())
      best_val_loss = float(lines[2].strip())
    return initial_epoch+1, wait, best_val_loss
  else:
    print("Tidak menemukan checkpoint. Mulai dari awal...")
    return 0, 0, float('inf')

## **FINE-TUNING IndoBERT**

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1",num_labels=label_strengthness_train.shape[1])

tf_model.h5:   0%|          | 0.00/656M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = Adam(learning_rate=3e-5)
loss = CategoricalCrossentropy(from_logits=True)

In [ ]:
tolerance = 3
min_delta = 1e-4 # berapa banyak perubahan agar training pada epoch tsb dinilai bagus
num_epoch = 3
initial_epoch, wait, best_loss = load_checkpoint(model, optimizer, checkpoint_dir)

for epoch in range(initial_epoch, num_epoch):
    print(f"Epoch {epoch+1}/{num_epoch}")
    print("-"*10)
    train_loss_total = 0
    train_step = 0

    for step, (x_batch_train, y_batch_train) in enumerate(formatted_strengthness_train):
      with tf.GradientTape() as tape:
        logits = model(x_batch_train, training=True).logits
        loss_value = loss(y_batch_train, logits)

      gradients = tape.gradient(loss_value, model.trainable_weights)
      optimizer.apply_gradients(zip(gradients, model.trainable_weights))

      train_loss_total += loss_value
      train_step += 1
      if train_step % 100 == 0:
        print(f"Training loss (step {step+1}): {loss_value:.4f}")
      train_loss_avg = train_loss_total / train_step

      val_loss_total = 0

    for x_batch_val, y_batch_val in formatted_strengthness_val:
      val_logits = model(x_batch_val, training=False).logits
      val_loss_value = loss(y_batch_val, val_logits)

      val_loss_total += val_loss_value
    val_loss_avg = val_loss_total / len(formatted_strengthness_val)

    print(f"Training loss: {train_loss_avg:.4f}")
    print(f"Validation loss: {val_loss_avg:.4f}")

    if val_loss_avg < best_loss - min_delta:
      best_loss = val_loss_avg
      wait = 0
      print("Loss Berkurang")
    else:
      wait += 1
      print ("Loss Meningkat")

    if wait >= tolerance:
      print("Early stopping triggered")
      break

    save_checkpoint(model, optimizer, epoch, wait, best_loss, checkpoint_filepath.format(epoch=epoch + 1))

Tidak menemukan checkpoint. Mulai dari awal...
Epoch 1/3
----------
Training loss (step 100): 0.3879
Training loss (step 200): 0.2014
Training loss (step 300): 0.9090
Training loss (step 400): 1.2558
Training loss: 0.6368
Validation loss: 0.5443
Loss Berkurang
Epoch 2/3
----------
Training loss (step 100): 0.6264
Training loss (step 200): 0.2245
Training loss (step 300): 0.3130
Training loss (step 400): 0.2751
Training loss: 0.3639
Validation loss: 0.5531
Loss Meningkat
Epoch 3/3
----------
Training loss (step 100): 0.0283
Training loss (step 200): 0.0970
Training loss (step 300): 0.0158
Training loss (step 400): 0.0235
Training loss: 0.2157
Validation loss: 0.7092
Loss Meningkat


## **EVALUATION**

In [ ]:
threshold = 0.5

preds = model.predict(formatted_strengthness_test)
probs = tf.nn.softmax(preds.logits, axis=-1).numpy()
preds_label = np.argmax(probs, axis=1)

true_labels = []
for _, label in formatted_strengthness_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

target_names = ['hs_weak', 'hs_moderate', 'hs_strong']

report = classification_report(true_labels, preds_label, target_names=target_names, zero_division=0)
accuracy = accuracy_score(true_labels, preds_label)

print(report)
print(f"Accuracy: {accuracy}")

24/24 [==============================] - 7s 39ms/step
              precision    recall  f1-score   support

     hs_weak       0.85      0.90      0.88        31
 hs_moderate       0.82      0.86      0.84        21
   hs_strong       1.00      0.85      0.92        20

    accuracy                           0.88        72
   macro avg       0.89      0.87      0.88        72
weighted avg       0.88      0.88      0.88        72

Accuracy: 0.875


## **SAVE MODEL AND TOKENIZER**

In [ ]:
model.save_pretrained('hs_strengthness_bert')
tokenizer.save_pretrained('hs_strengthness_bert')

shutil.make_archive('hs_strengthness_bert', 'zip', 'hs_multilabel_bert')
files.download('hs_strengthness_bert.zip')

## **EXAMPLE**

In [ ]:
def preprocess_input(text):
    inputs = tokenizer(text, return_tensors="tf", padding=True, truncation=True, max_length=512)
    return inputs

sentence = "ganyang fufufafa"
inputs = preprocess_input(sentence)


In [ ]:
predictions = model(inputs)[0]

probabilities = tf.nn.softmax(predictions, axis=-1)
predicted_class = np.argmax(probabilities, axis=-1)

print(f"Prediksi kelas: {predicted_class[0]}")

Prediksi kelas: 0


In [ ]:
one_hot_label = np.zeros(3)
one_hot_label[predicted_class] = 1

print(f"OneHotEncoding Prediksi: {one_hot_label}")

OneHotEncoding Prediksi: [1. 0. 0.]
